In [ ]:
import flwr as fl
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Carica e preprocessa il dataset MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train.reshape((-1, 28 * 28))
x_test = x_test.reshape((-1, 28 * 28))

# Definisce una semplice MLP
def create_model():
    model = keras.Sequential([
        layers.Input(shape=(28 * 28,)),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Crea il modello
model = create_model()

class MyClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        
        return model.get_weights()

    def set_parameters(self, parameters):
        
        model.set_weights(parameters)

    def fit(self, parameters, config):
        
        self.set_parameters(parameters)
        history = model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=1)
        num_examples = x_train.shape[0]
        # Ottiene l'accuracy dell'ultima epoca
        final_accuracy = history.history["accuracy"][-1]
        return model.get_weights(), num_examples, {"accuracy": final_accuracy}

    def evaluate(self, parameters, config):
        
        self.set_parameters(parameters)
        loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
        num_examples = x_test.shape[0]
        return loss, num_examples, {"accuracy": accuracy}

if __name__ == "__main__":
    # Connetti il client al server (modifica l'indirizzo se necessario)
    fl.client.start_numpy_client(server_address="localhost:8080", client=MyClient())


INFO flwr 2025-03-11 13:23:10,887 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2025-03-11 13:23:10,890 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2025-03-11 13:23:10,893 | connection.py:42 | ChannelConnectivity.READY


Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2557 - accuracy: 0.9275
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1123 - accuracy: 0.9669
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0761 - accuracy: 0.9775
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0573 - accuracy: 0.9823
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0438 - accuracy: 0.9861
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' objec

DEBUG flwr 2025-03-11 13:23:44,183 | connection.py:139 | gRPC channel closed
INFO flwr 2025-03-11 13:23:44,183 | app.py:215 | Disconnect and shut down
